In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from tabpfn import TabPFNClassifier


In [31]:
# ƒê·ªçc file CSV
df = pd.read_csv(r"D:\MINI_Project\DoAn3\perdict_train_XGBoost\students_grading_dataset_clean.csv")

# Lo·∫°i b·ªè c√°c c·ªôt kh√¥ng c·∫ßn thi·∫øt
df.drop(['Student_ID', 'First_Name', 'Last_Name', 'Email',
         'Midterm_Score', 'Final_Score', 'Projects_Score', 'Total_Score'],
        axis=1, inplace=True)

# M√£ ho√° nh√£n ph√¢n lo·∫°i trong c√°c c·ªôt d·∫°ng object
for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col])

# M√£ ho√° nh√£n 'Grade'
label_encoder = LabelEncoder()
df['Grade'] = label_encoder.fit_transform(df['Grade'])  # A=0, B=1, C=2, D=3


In [32]:
X = df.drop('Grade', axis=1).values
y = df['Grade'].values

# Chia t·∫≠p train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [33]:
from sklearn.utils import shuffle
import numpy as np

# --- L·∫•y ch·ªâ s·ªë t·ª´ng l·ªõp ---
idx_A = np.where(y_train == 0)[0]  # A
idx_B = np.where(y_train == 1)[0]  # B
idx_C = np.where(y_train == 2)[0]  # C
idx_D = np.where(y_train == 3)[0]  # D

# --- Gi·ªØ to√†n b·ªô A v√† D ---
X_A, y_A = X_train[idx_A], y_train[idx_A]
X_D, y_D = X_train[idx_D], y_train[idx_D]

# --- L·∫•y ng·∫´u nhi√™n 300 m·∫´u t·ª´ B v√† C ---
np.random.seed(42)
idx_B_sample = np.random.choice(idx_B, size=300, replace=False)
idx_C_sample = np.random.choice(idx_C, size=300, replace=False)

X_B, y_B = X_train[idx_B_sample], y_train[idx_B_sample]
X_C, y_C = X_train[idx_C_sample], y_train[idx_C_sample]

# --- G·ªôp l·∫°i t·∫≠p nh·ªè ---
X_sample = np.vstack([X_A, X_B, X_C, X_D])
y_sample = np.hstack([y_A, y_B, y_C, y_D])

# --- Tr·ªôn d·ªØ li·ªáu ---
X_sample, y_sample = shuffle(X_sample, y_sample, random_state=42)

# --- TƒÉng tr·ªçng s·ªë cho A v√† D (l·ªõp 0 v√† 3) ---
# B·∫°n c√≥ th·ªÉ t√πy ch·ªânh c√°c gi√° tr·ªã b√™n d∆∞·ªõi
class_weights = {
    0: 3.0,   # L·ªõp A (√≠t h∆°n l·ªõp B & C)
    1: 1.0,   # L·ªõp B
    2: 1.0,   # L·ªõp C
    3: 5.0    # L·ªõp D (r·∫•t √≠t)
}



In [34]:
# In s·ªë l∆∞·ª£ng t·ª´ng l·ªõp sau khi ch·ªçn m·∫´u
unique, counts = np.unique(y_sample, return_counts=True)
for label, count in zip(unique, counts):
    print(f"L·ªõp {label_encoder.inverse_transform([label])[0]}: {count} m·∫´u")


L·ªõp 0: 122 m·∫´u
L·ªõp 1: 300 m·∫´u
L·ªõp 2: 300 m·∫´u
L·ªõp 3: 34 m·∫´u


In [35]:
# Kh·ªüi t·∫°o m√¥ h√¨nh TabPFN (CPU)
model = TabPFNClassifier(
    device='cpu',
    ignore_pretraining_limits=True  # Cho ph√©p train t·∫≠p >1000 d√≤ng tr√™n CPU
)

# Hu·∫•n luy·ªán
model.fit(X_sample, y_sample)


TabPFNClassifier(device='cpu', ignore_pretraining_limits=True)

In [36]:
# ====== D·ª± ƒëo√°n tr√™n t·∫≠p test ======
y_pred = model.predict(X_test)  # D·ª± ƒëo√°n nh√£n cho d·ªØ li·ªáu ki·ªÉm tra

# ====== In b√°o c√°o ph√¢n lo·∫°i ======
from sklearn.metrics import classification_report

print("\n=== üìä Classification Report ===")
print(classification_report(
    y_test,                      # Nh√£n th·ª±c t·∫ø
    y_pred,                      # Nh√£n d·ª± ƒëo√°n
    #target_names=label_encoder.classes_  # Hi·ªÉn th·ªã t√™n l·ªõp g·ªëc (A, B, C, D)
))



=== üìä Classification Report ===
              precision    recall  f1-score   support

           0       0.12      0.32      0.18        31
           1       0.63      0.50      0.56       562
           2       0.51      0.61      0.56       399
           3       0.00      0.00      0.00         8

    accuracy                           0.53      1000
   macro avg       0.32      0.36      0.32      1000
weighted avg       0.56      0.53      0.54      1000

